In [1]:
import numpy as np
import pandas as pd
import pprint
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

pd.options.display.float_format = '{:.2f}'.format

### <font color="orange">  **Data Loading and Visualizaiton** </font>

In [2]:
df = pd.read_csv('health_insurance_train.csv')
df_autograder = pd.read_csv('health_insurance_autograde.csv')
                            
display(df.iloc[8:16])
display(df_autograder.iloc[8:16])

,whrswk,hhi,whi,hhi2,education,race,hispanic,experience,kidslt6,kids618,husby,region
8,5.00,yes,no,yes,13-15years,NaN,no,20.00,1.00,1.00,53.00,south
9,40.00,no,yes,yes,12years,white,no,9.00,1.00,1.00,18.00,west
10,40.00,no,no,no,12years,white,no,33.00,0.00,0.00,0.00,other
11,40.00,no,no,no,9-11years,white,no,5.50,1.00,0.00,20.00,northcentral
12,35.00,no,yes,no,12years,NaN,yes,26.00,0.00,1.00,0.00,south
13,45.00,no,yes,yes,16years,white,no,7.00,0.00,0.00,40.00,south
14,0.00,yes,no,yes,12years,white,no,38.00,0.00,0.00,0.00,other
15,20.00,yes,no,yes,12years,white,no,9.00,1.00,1.00,33.00,south


,hhi,whi,hhi2,education,race,hispanic,experience,kidslt6,kids618,husby,region
8,yes,yes,yes,13-15years,white,no,35.00,0.00,0.00,52.86,south
9,no,yes,no,16years,white,no,1.00,0.00,0.00,5.00,northcentral
10,no,no,no,12years,white,no,14.00,NaN,NaN,0.00,northcentral
11,no,yes,no,12years,white,no,44.00,NaN,NaN,0.00,west
12,no,yes,yes,13-15years,white,no,23.00,0.00,3.00,45.00,west
13,no,yes,no,16years,white,no,10.00,0.00,0.00,0.00,south
14,yes,no,yes,>16years,white,no,23.00,0.00,0.00,40.10,west
15,yes,no,yes,12years,white,no,22.00,1.00,0.00,24.85,other


In [3]:
''''
* Data visualization (Edlyn)
- Make nice histograms of the features and the target
- Make a covariance matrix of the features
'''

"'\n* Data visualization (Edlyn)\n- Make nice histograms of the features and the target\n- Make a covariance matrix of the features\n"

### <font color="orange">  **Data processing fucntions** </font>

In [4]:
''''
* Create the data pipeline functions (Edlyn)

- Preprocess data (PIPE 1, PIPE 2)
    . Apply one hot encoding for region, race and hispanic. Make a column for race_nan and hispanic_nan. 
      People that decide to not fill these information might have a characteristic profile. So we can let the
      model decide if it is important or not.

    . Make remaining yes/no 1 and -1 (helps KNN, since the features are logically opoosite of each other)

    . Make True/False 1 and 0 (When it is not just about "yes/no" give 1 to True and 0 to False)

    . Process education column, nan becomes average 

    . You can apply remove first if you want


- Regular scaling (PIPE 1) 

- Selective scaling (PIPE 2)

- Data filtering with mahalanobis (PIPE 2)

'''


'\'\n* Create the data pipeline functions (Edlyn)\n\n- Preprocess data (PIPE 1, PIPE 2)\n    . Apply one hot encoding for region, race and hispanic. Make a column for race_nan and hispanic_nan. \n      People that decide to not fill these information might have a characteristic profile. So we can let the\n      model decide if it is important or not.\n\n    . Make remaining yes/no 1 and -1 (helps KNN, since the features are logically opoosite of each other)\n\n    . Make True/False 1 and 0 (When it is not just about "yes/no" give 1 to True and 0 to False)\n\n    . Process education column, nan becomes average \n\n    . You can apply remove first if you want, but I we should verify if it helps\n\n\n- Regular scaling (PIPE 1) \n\n- Selective scaling (PIPE 2)\n\n- Data filtering with mahalanobis (PIPE 2)\n\n'

### <font color="orange">  **Pipeline creation** </font>

In [5]:
df_1 = pd.DataFrame(np.random.rand(40, 5)) # Use the pipiline 1 function

display(df_1.iloc[9:30])

,0,1,2,3,4
9,0.11,0.44,0.89,0.07,0.92
10,0.64,0.33,0.75,0.61,0.18
11,0.77,0.05,0.91,0.90,0.45
12,0.11,0.66,0.60,0.84,0.51
13,0.88,0.19,0.77,0.71,0.21
14,0.08,0.10,0.04,0.51,0.75
15,0.71,0.01,0.38,0.75,0.56
16,0.72,0.56,0.59,0.87,0.08
17,0.60,0.49,0.52,0.67,0.24
18,0.48,0.87,0.90,0.13,0.32


In [6]:
df_2 = pd.DataFrame(np.random.rand(40, 5)) # Use the pipiline 2 function

display(df_2.iloc[9:30])

,0,1,2,3,4
9,0.84,0.58,0.41,0.27,0.50
10,0.00,0.40,0.96,0.74,0.26
11,0.23,0.49,0.41,0.76,0.19
12,0.57,0.51,0.70,0.91,0.43
13,0.47,0.18,0.55,0.36,0.27
14,0.16,0.51,0.90,0.30,0.40
15,0.67,0.72,0.32,0.18,0.06
16,0.21,0.22,0.94,0.57,0.38
17,0.82,0.68,0.41,0.54,0.10
18,0.92,0.91,0.17,0.43,0.74


In [7]:
# Split the data into seen and unseen while keeping it as a pandas dataframe
fraction = 0.2  # 20% of the rows

#-------------------PIPE 1-------------------
df_unseen_1 = df_1.sample(frac = fraction, random_state=42) # Get 20% of random rows
df_seen_1 = df_1.drop(df_unseen_1.index) # Get the remaining 80% of the rows

X_seen_1 = df_1.iloc[:, 1:]
Y_seen_1 = df_1.iloc[:, 0]

X_unseen_1 = df_unseen_1.iloc[:, 1:]
Y_unseen_1 = df_unseen_1.iloc[:, 0]

#-------------------PIPE 2-------------------
df_unseen_2 = df_2.sample(frac = fraction, random_state=42)
df_seen_2 = df_2.drop(df_unseen_2.index)

X_seen_2 = df_2.iloc[:, 1:]
Y_seen_2 = df_2.iloc[:, 0]

X_unseen_2 = df_unseen_2.iloc[:, 1:]
Y_unseen_2 = df_unseen_2.iloc[:, 0]

### <font color="orange">  **Training Functions** </font>

In [8]:

#-------------------Dummy-------------------
from sklearn.dummy import DummyRegressor
def train_dummy_predictor(X, Y):
    model = DummyRegressor(strategy='mean')
    model.fit(X, Y)
    return model

#-------------------KNN-------------------
from sklearn.neighbors import KNeighborsRegressor
def train_knn_regressor(X, Y, param_grid):
    model = KNeighborsRegressor(**param_grid)
    model.fit(X, Y)
    Y_pred = model.predict(X)
    loss_values = [mean_absolute_error(Y, Y_pred)]
    return model,loss_values

#-------------------SGD-------------------
from sklearn.linear_model import SGDRegressor
def train_sgd_regressor(X, Y, params):
    model = SGDRegressor(**params)
    epochs = params['max_iter']

    loss_values = []
    for epoch in range(epochs):
        model.partial_fit(X, Y)
        Y_pred = model.predict(X)
        epoch_loss = mean_absolute_error(Y, Y_pred)
        loss_values.append(epoch_loss)
    
    return model, loss_values

#-----------Decision Tree-------------------
from sklearn.tree import DecisionTreeRegressor
def train_decision_tree_regressor(X, Y, params):

    '''params : dict
        Dictionary of parameters to pass to DecisionTreeRegressor.'''
    
    # splitter = Supported strategies are "best" to choose the best split and "random" to choose the best random split.
    # max_features = The number of features to consider when looking for the best split
    # min_samples_split = The minimum number of samples required to split an internal node
    # min_samples_leaf = The minimum number of samples required to be at a leaf node

    model = DecisionTreeRegressor(**params,random_state = 42)
    loss_values = []
    
    # Custom training loop with logging
    for depth in range(1, params['max_depth'] + 1):
        model.set_params(max_depth=depth)
        model.fit(X, Y)
        Y_pred = model.predict(X)
        loss = mean_absolute_error(Y, Y_pred)
        loss_values.append(loss)
    
    return model, loss_values


### <font color="orange">  **Create model dict and test dataframe** </font>

In [11]:

#----------Creation of models dict-----------
models_dict = {'KNN': None, 'SGD': None, 'Tree':None}
for key in models_dict:
    models_dict[key] = {'defalt' :None, 'best_param':None, 'best_model' :None, 'ensemble':None, 'best_ensemble':None}

pprint.pprint(models_dict)

#----------Creation of test dfs-----------
test_df_1 = pd.DataFrame(index=['UT','T','ET','BET','BETA'],columns=['KNN','SGD','Tree'])
test_df_2 = pd.DataFrame(index=['UT','T','ET','BET','BETA'],columns=['KNN','SGD','Tree'])

display(test_df_1)


{'KNN': {'best_ensemble': None,
         'best_model': None,
         'best_param': None,
         'defalt': None,
         'ensemble': None},
 'SGD': {'best_ensemble': None,
         'best_model': None,
         'best_param': None,
         'defalt': None,
         'ensemble': None},
 'Tree': {'best_ensemble': None,
          'best_model': None,
          'best_param': None,
          'defalt': None,
          'ensemble': None}}


,KNN,SGD,Tree
UT,NaN,NaN,NaN
T,NaN,NaN,NaN
ET,NaN,NaN,NaN
BET,NaN,NaN,NaN
BETA,NaN,NaN,NaN
